In [1]:
%pip install pdfservices-sdk==2.2.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
import os.path
import pandas as pd
import json

from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_renditions_element_type import \
    ExtractRenditionsElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
import zipfile

logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

In [3]:
def extractPDF(file_name, output_folder):
    try:
    # get base path.
    # base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

    # Initial setup, create credentials instance.
        credentials = Credentials.service_account_credentials_builder() \
            .from_file("C:/Users/asus/Desktop/adobe_papyrus_nebulae/pdfservices-api-credentials.json") \
            .build()

        # Create an ExecutionContext using credentials and create a new operation instance.
        execution_context = ExecutionContext.create(credentials)
        extract_pdf_operation = ExtractPDFOperation.create_new()

        # Set operation input from a source file.
        source = FileRef.create_from_local_file("C:/Users/asus/Desktop/adobe_papyrus_nebulae/resources/" + file_name)
        extract_pdf_operation.set_input(source)

        # Build ExtractPDF options and set them into the operation
        extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
            .with_elements_to_extract([ExtractElementType.TEXT, ExtractElementType.TABLES]) \
            .with_elements_to_extract_renditions([ExtractRenditionsElementType.TABLES,
                                                ExtractRenditionsElementType.FIGURES]) \
            .build()
        extract_pdf_operation.set_options(extract_pdf_options)

        # Execute the operation.
        result: FileRef = extract_pdf_operation.execute(execution_context)

        # Save the result to the specified location.
        result.save_as("C:/Users/asus/Desktop/adobe_papyrus_nebulae/" + output_folder + "/ExtractTextTableWithFigureTableRendition.zip")
        with zipfile.ZipFile("C:/Users/asus/Desktop/adobe_papyrus_nebulae/" + output_folder + "/ExtractTextTableWithFigureTableRendition.zip", 'r') as zip_ref:
            zip_ref.extractall("C:/Users/asus/Desktop/adobe_papyrus_nebulae/" + output_folder)
        output = pd.read_excel('C:/Users/asus/Desktop/adobe_papyrus_nebulae/' + output_folder + '/tables/fileoutpart0.xlsx')
        print(output)
    except (ServiceApiException, ServiceUsageException, SdkException):
        logging.exception("Exception encountered while executing operation")

In [4]:
extractPDF('output8.pdf','output8')

INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:All validations successfully done. Beginning ExtractPDF operation execution
INFO:root:Downloading file to C:\Users\asus\AppData\Local\Temp\sdk_result\b6c9622d137311ee9fca9bbb70d9d758.zip
INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:Extract Operation Successful - Transaction ID: 3fb504fd-3575-4017-859a-23ada4054cfb
INFO:adobe.pdfservices.operation.internal.io.file_ref_impl:Moving file at C:\Users\asus\AppData\Local\Temp\sdk_result\b6c9622d137311ee9fca9bbb70d9d758.zip to target C:/Users/asus/Desktop/adobe_papyrus_nebulae/output8/ExtractTextTableWithFigureTableRendition.zip


Empty DataFrame
Columns: [ITEM _x000D_, QTY _x000D_, RATE _x000D_, AMOUNT _x000D_]
Index: []


In [5]:
extractPDF('output9.pdf','output9')

INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:All validations successfully done. Beginning ExtractPDF operation execution
ERROR:root:Exception encountered while executing operation
Traceback (most recent call last):
  File "c:\Users\asus\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "c:\Users\asus\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _validate_conn
    conn.connect()
  File "c:\Users\asus\anaconda3\lib\site-packages\urllib3\connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "c:\Users\asus\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "c:\Users\asus\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "c:\Users\asus\anaconda3\lib\ssl.py", line 500, in

In [118]:
def preprocess(output_folder):
    table1 = pd.read_excel('C:/Users/asus/Desktop/adobe_papyrus_nebulae/'+ output_folder +'/tables/fileoutpart0.xlsx', header=None, names=['description'])
    table1.replace(regex=['_x000D_'],value='',inplace=True)
    table3 = pd.read_excel('C:/Users/asus/Desktop/adobe_papyrus_nebulae/'+ output_folder +'/tables/fileoutpart4.xlsx' , header=None, names=['item','qty','rate','amount'])
    table3.replace(regex=['_x000D_'],value='',inplace=True)
    table3['amount'] = table3['amount'].str.replace('$','', regex=True)
    table3[['qty', 'rate','amount']] = table3[['qty', 'rate','amount']].apply(pd.to_numeric)
    f = open('C:/Users/asus/Desktop/adobe_papyrus_nebulae/'+ output_folder +'/structuredData.json')
    json_data = json.load(f)
    text_data = []
    for i in json_data['elements']:
        try:
            # print(i["Text"])
            text_data.append(i["Text"])
        except:
            print(end='')
    print(text_data)
    f.close()
    table4 = pd.read_excel('C:/Users/asus/Desktop/adobe_papyrus_nebulae/'+ output_folder +'/tables/fileoutpart6.xlsx', header=None, names=['type','amount'])
    table4.replace(regex=['_x000D_'],value='',inplace=True)
    table4['amount'] = table4['amount'].str.replace('$','', regex=True)
    table4[['amount']] = table4[['amount']].apply(pd.to_numeric)
    # final_data = {
    #     'Bussiness__City' : text_data[2].split(',')[0],
    #     'Bussiness__Country': text_data[2].split(',')[1],
    #     'Bussiness__Description': text_data[8],
    #     'Bussiness__Name': text_data[0],
    #     'Bussiness__StreetAddress': text_data[1],
    #     'Bussiness__Zipcode': text_data[3],
    #     'Customer__Address__line1': text_data[14],
    #     'Customer__Address__line2': text_data[15],
    #     'Customer__Email': text_data[11] + text_data[12],
    #     'Customer__Name' : text_data[10],
    #     'Customer__PhoneNumber': text_data[13],
    #     'Invoice__BillDetails__Name': table3['item'],
    #     'Invoice__BillDetails__Quantity': table3['qty'],
    #     'Invoice__BillDetails__Rate': table3['rate'],
    #     'Invoice__Description': table1['description'].str.cat(sep=' '),
    #     'Invoice__DueDate': text_data[21][10:-1],
    #     'Invoice__IssueDate': text_data[6][:-1],
    #     'Invoice__Number': text_data[4][8:-1],
    #     'Invoice__Tax': table4[table4['type'].str.contains('Tax %')]['amount'][1]
    # }
    # final_df = pd.DataFrame(final_data)
    return text_data

In [113]:
preprocess('output0')

NearBy Electronics 
3741 Glory Road, Jamestown, 
Tennessee, USA 
38556 
Invoice# NL57EPAS7793742478 
Issue date 
12-05-2023 
NearBy Electronics 
We are here to serve you better. Reach out to us in case of any concern or feedbacks. 
BILL TO 
Willis Koelpin 
Willis_Koelpin4@yahoo.co 
m 
783-402-5895 
353 Cara Shoals 
Suchitlán 
DETAILS 
NULL
NULL
minim velit velit fugiat culpa 
NULL
deserunt ex aliquip cillum est 
NULL
aliqua ex amet amet 
PAYMENT 
Due date: 08-07-2023 
$22337.7 
NULL
NULL
ITEM 
NULL
QTY 
NULL
RATE 
NULL
AMOUNT 
NULL
NULL
Rustic Rubber Gloves 
NULL
102 
NULL
29 
NULL
$2958 
NULL
Fantastic Granite Salad 
NULL
39 
NULL
27 
NULL
$1053 
NULL
Small Fresh Salad 
NULL
95 
NULL
69 
NULL
$6555 
NULL
Fantastic Metal Chips 
NULL
67 
NULL
49 
NULL
$3283 
NULL
Refined Cotton Pants 
NULL
79 
NULL
72 
NULL
$5688 
NULL
Ergonomic Concrete Towels 
NULL
35 
NULL
22 
NULL
$770 
NULL
NULL
Subtotal 
NULL
$20307 
NULL
Tax % 
NULL
10 
Total Due 
$22337.7 
 NearBy Electronics  3741 Glory Road, J

In [108]:
final_df0

,Bussiness__City,Bussiness__Country,Bussiness__Description,Bussiness__Name,Bussiness__StreetAddress,Bussiness__Zipcode,Customer__Address__line1,Customer__Address__line2,Customer__Email,Customer__Name,Customer__PhoneNumber,Invoice__BillDetails__Name,Invoice__BillDetails__Quantity,Invoice__BillDetails__Rate,Invoice__Description,Invoice__DueDate,Invoice__IssueDate,Invoice__Number,Invoice__Tax
0,Tennessee,USA,We are here to serve you better. Reach out to ...,NearBy Electronics,"3741 Glory Road, Jamestown,",38556,353 Cara Shoals,Suchitlán,Willis_Koelpin4@yahoo.co m,Willis Koelpin,783-402-5895,Rustic Rubber Gloves,102,29,minim velit velit fugiat culpa deserunt ex al...,08-07-2023,12-05-2023,NL57EPAS7793742478,10
1,Tennessee,USA,We are here to serve you better. Reach out to ...,NearBy Electronics,"3741 Glory Road, Jamestown,",38556,353 Cara Shoals,Suchitlán,Willis_Koelpin4@yahoo.co m,Willis Koelpin,783-402-5895,Fantastic Granite Salad,39,27,minim velit velit fugiat culpa deserunt ex al...,08-07-2023,12-05-2023,NL57EPAS7793742478,10
2,Tennessee,USA,We are here to serve you better. Reach out to ...,NearBy Electronics,"3741 Glory Road, Jamestown,",38556,353 Cara Shoals,Suchitlán,Willis_Koelpin4@yahoo.co m,Willis Koelpin,783-402-5895,Small Fresh Salad,95,69,minim velit velit fugiat culpa deserunt ex al...,08-07-2023,12-05-2023,NL57EPAS7793742478,10
3,Tennessee,USA,We are here to serve you better. Reach out to ...,NearBy Electronics,"3741 Glory Road, Jamestown,",38556,353 Cara Shoals,Suchitlán,Willis_Koelpin4@yahoo.co m,Willis Koelpin,783-402-5895,Fantastic Metal Chips,67,49,minim velit velit fugiat culpa deserunt ex al...,08-07-2023,12-05-2023,NL57EPAS7793742478,10
4,Tennessee,USA,We are here to serve you better. Reach out to ...,NearBy Electronics,"3741 Glory Road, Jamestown,",38556,353 Cara Shoals,Suchitlán,Willis_Koelpin4@yahoo.co m,Willis Koelpin,783-402-5895,Refined Cotton Pants,79,72,minim velit velit fugiat culpa deserunt ex al...,08-07-2023,12-05-2023,NL57EPAS7793742478,10
5,Tennessee,USA,We are here to serve you better. Reach out to ...,NearBy Electronics,"3741 Glory Road, Jamestown,",38556,353 Cara Shoals,Suchitlán,Willis_Koelpin4@yahoo.co m,Willis Koelpin,783-402-5895,Ergonomic Concrete Towels,35,22,minim velit velit fugiat culpa deserunt ex al...,08-07-2023,12-05-2023,NL57EPAS7793742478,10


In [109]:
extractPDF('output1.pdf','output1')

INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:All validations successfully done. Beginning ExtractPDF operation execution
INFO:root:Downloading file to C:\Users\asus\AppData\Local\Temp\sdk_result\89d19ba410d311eeb8839bbb70d9d758.zip
INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:Extract Operation Successful - Transaction ID: ae2fee8c-67ca-4d0c-a78c-8dac3de8d920
INFO:adobe.pdfservices.operation.internal.io.file_ref_impl:Moving file at C:\Users\asus\AppData\Local\Temp\sdk_result\89d19ba410d311eeb8839bbb70d9d758.zip to target C:/Users/asus/Desktop/adobe_papyrus_nebulae/output1/ExtractTextTableWithFigureTableRendition.zip


                                     BILL TO _x000D_ DETAILS _x000D_  \
0  Don Zulauf Don_Zulauf40@hotmail.co m 110-825-4...  veniam _x000D_   

                                PAYMENT _x000D_  
0  Due date: 08-07-2023 _x000D_$49432.9 _x000D_  


In [119]:
out = preprocess('output1')

['NearBy Electronics ', '3741 Glory Road, Jamestown, Tennessee, USA ', '38556 ', 'Invoice# TG67TS4201156943079376302586 Issue date 12-05-2023 ', 'NearBy Electronics ', 'We are here to serve you better. Reach out to us in case of any concern or feedbacks. ', 'BILL TO ', 'DETAILS ', 'PAYMENT ', 'Don Zulauf ', 'Don_Zulauf40@hotmail.co ', 'm ', '110-825-4839 ', '121 Jacobs River ', 'Port-Vila ', 'veniam ', 'Due date: 08-07-2023 ', '$49432.9 ', 'ITEM ', 'QTY ', 'RATE ', 'AMOUNT ', 'Awesome Fresh Hat ', '28 ', '60 ', '$1680 ', 'Licensed Fresh Pizza ', '114 ', '94 ', '$10716 ', 'Unbranded Concrete Shirt ', '44 ', '95 ', '$4180 ', 'Sleek Plastic Shirt ', '28 ', '96 ', '$2688 ', 'Rustic Plastic Fish ', '83 ', '44 ', '$3652 ', 'Intelligent Granite Computer ', '117 ', '85 ', '$9945 ', 'Ergonomic Fresh Salad ', '34 ', '25 ', '$850 ', 'Generic Wooden Sausages ', '128 ', '87 ', '$11136 ', 'Handmade Wooden Shoes ', '4 ', '23 ', '$92 ', 'Subtotal ', '$44939 ', 'Tax % ', '10 ', 'Total Due ', '$49432.9 

In [127]:
len(out)

64

In [130]:
def extractPDF2(input_file, output_folder):
    try:
    # get base path.
        # base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

        # Initial setup, create credentials instance.
        credentials = Credentials.service_account_credentials_builder() \
            .from_file("C:/Users/asus/Desktop/adobe_papyrus_nebulae/pdfservices-api-credentials.json") \
            .build()

        # Create an ExecutionContext using credentials and create a new operation instance.
        execution_context = ExecutionContext.create(credentials)
        extract_pdf_operation = ExtractPDFOperation.create_new()

        # Set operation input from a source file.
        source = FileRef.create_from_local_file("C:/Users/asus/Desktop/adobe_papyrus_nebulae/resources/" + input_file)
        extract_pdf_operation.set_input(source)

        # Build ExtractPDF options and set them into the operation
        # extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
        #     .with_elements_to_extract([ExtractElementType.TEXT, ExtractElementType.TABLES]) \
        #     .with_element_to_extract_renditions(ExtractRenditionsElementType.TABLES) \
        #     .with_table_structure_format(TableStructureType.CSV) \
        #     .build()
        extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
        .with_element_to_extract(ExtractElementType.TEXT) \
        .build()
        extract_pdf_operation.set_options(extract_pdf_options)

        # Execute the operation.
        result: FileRef = extract_pdf_operation.execute(execution_context)

        # Save the result to the specified location.
        result.save_as("C:/Users/asus/Desktop/adobe_papyrus_nebulae/" + output_folder + "/ExtractTextTableWithFigureTableRendition.zip")
        with zipfile.ZipFile("C:/Users/asus/Desktop/adobe_papyrus_nebulae/" + output_folder + "/ExtractTextTableWithFigureTableRendition.zip", 'r') as zip_ref:
            zip_ref.extractall("C:/Users/asus/Desktop/adobe_papyrus_nebulae/" + output_folder)
        # output = pd.read_excel('C:/Users/asus/Desktop/adobe_papyrus_nebulae/' + output_folder + '/tables/fileoutpart0.xlsx')
        # print(output)
    except (ServiceApiException, ServiceUsageException, SdkException):
        logging.exception("Exception encountered while executing operation")

In [132]:
extractPDF2('output1.pdf','output1_2')

INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:All validations successfully done. Beginning ExtractPDF operation execution
INFO:root:Downloading file to C:\Users\asus\AppData\Local\Temp\sdk_result\d75c4cd1111811eea24f9bbb70d9d758.zip
INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:Extract Operation Successful - Transaction ID: a87c5c4b-eb2f-45ec-965f-20bebfeb47ad
INFO:adobe.pdfservices.operation.internal.io.file_ref_impl:Moving file at C:\Users\asus\AppData\Local\Temp\sdk_result\d75c4cd1111811eea24f9bbb70d9d758.zip to target C:/Users/asus/Desktop/adobe_papyrus_nebulae/output1_2/ExtractTextTableWithFigureTableRendition.zip


In [137]:
def postprocess(output_folder):
    f = open('C:/Users/asus/Desktop/adobe_papyrus_nebulae/'+ output_folder +'/structuredData.json')
    json_data = json.load(f)
    text_data = ''
    for i in json_data['elements']:
        try:
            # print(i["Text"])
            # text_data.append(i["Text"])
            text_data = text_data + i["Text"]
        except:
            print(end='')
    print(text_data)
    f.close()
    return text_data

In [138]:
out = postprocess('output1_2')

NearBy Electronics 3741 Glory Road, Jamestown, Tennessee, USA 38556 Invoice# TG67TS4201156943079376302586 Issue date 12-05-2023 NearBy Electronics We are here to serve you better. Reach out to us in case of any concern or feedbacks. BILL TO DETAILS PAYMENT Don Zulauf Don_Zulauf40@hotmail.co m 110-825-4839 121 Jacobs River Port-Vila veniam Due date: 08-07-2023 $49432.9 ITEM QTY RATE AMOUNT Awesome Fresh Hat 28 60 $1680 Licensed Fresh Pizza 114 94 $10716 Unbranded Concrete Shirt 44 95 $4180 Sleek Plastic Shirt 28 96 $2688 Rustic Plastic Fish 83 44 $3652 Intelligent Granite Computer 117 85 $9945 Ergonomic Fresh Salad 34 25 $850 Generic Wooden Sausages 128 87 $11136 Handmade Wooden Shoes 4 23 $92 Subtotal $44939 Tax % 10 Total Due $49432.9 


In [135]:
extractPDF2('output0.pdf','output0_2')

INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:All validations successfully done. Beginning ExtractPDF operation execution
INFO:root:Downloading file to C:\Users\asus\AppData\Local\Temp\sdk_result\47f3a707111911ee8edd9bbb70d9d758.zip
INFO:adobe.pdfservices.operation.pdfops.extract_pdf_operation:Extract Operation Successful - Transaction ID: 96dddab8-ad2c-4503-8d23-bf45a22dee61
INFO:adobe.pdfservices.operation.internal.io.file_ref_impl:Moving file at C:\Users\asus\AppData\Local\Temp\sdk_result\47f3a707111911ee8edd9bbb70d9d758.zip to target C:/Users/asus/Desktop/adobe_papyrus_nebulae/output0_2/ExtractTextTableWithFigureTableRendition.zip


In [139]:
out = postprocess('output0_2')

NearBy Electronics 3741 Glory Road, Jamestown, Tennessee, USA 38556 Invoice# NL57EPAS7793742478 Issue date 12-05-2023 NearBy Electronics We are here to serve you better. Reach out to us in case of any concern or feedbacks. BILL TO Willis Koelpin Willis_Koelpin4@yahoo.co m 783-402-5895 353 Cara Shoals Suchitlán DETAILS minim velit velit fugiat culpa deserunt ex aliquip cillum est aliqua ex amet amet PAYMENT Due date: 08-07-2023 $22337.7 ITEM QTY RATE AMOUNT Rustic Rubber Gloves 102 29 $2958 Fantastic Granite Salad 39 27 $1053 Small Fresh Salad 95 69 $6555 Fantastic Metal Chips 67 49 $3283 Refined Cotton Pants 79 72 $5688 Ergonomic Concrete Towels 35 22 $770 Subtotal $20307 Tax % 10 Total Due $22337.7 
